In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [13]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [17]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(f"logits are {logits}")
pred_probab = nn.Softmax(dim=1)(logits)
print(f"pred_probab are {pred_probab}")
y_pred = pred_probab.argmax(1)
print(f"predicted class {y_pred}")


logits are tensor([[-0.0566, -0.0103,  0.0108, -0.0983, -0.0551,  0.0609,  0.0493,  0.1488,
         -0.0039, -0.0155]], device='mps:0', grad_fn=<LinearBackward0>)
pred_probab are tensor([[0.0940, 0.0985, 0.1006, 0.0902, 0.0941, 0.1057, 0.1045, 0.1154, 0.0991,
         0.0979]], device='mps:0', grad_fn=<SoftmaxBackward0>)
predicted class tensor([7], device='mps:0')


In [20]:
#example to play with softmax, dim=1 is across columns so for an example. 
#dim=0 would be across a feature which might not be useful for classification in usual sense
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
print (input)
output = m(input)
print (output)

tensor([[ 0.0750,  0.5559, -1.4322],
        [-1.2489,  0.6642, -0.7276]])
tensor([[0.3522, 0.5697, 0.0780],
        [0.1057, 0.7162, 0.1781]])


In [21]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [22]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [23]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [24]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4191, -0.2394,  0.1291,  0.0058, -0.0968, -0.4242, -0.2887, -0.6720,
          0.3976,  0.0887,  0.2234,  0.1923,  0.1938,  0.4474,  0.0274,  0.0707,
          0.5935, -0.0646,  0.1933,  0.3510],
        [-0.2446,  0.0357,  0.3397,  0.1080, -0.3345, -0.4086, -0.2138, -0.5635,
          0.1673,  0.0475,  0.1926,  0.0749,  0.2884,  0.4969, -0.2151,  0.3091,
          0.8137, -0.1745, -0.1847,  0.3526],
        [-0.3357, -0.3323,  0.2898,  0.0554, -0.1658, -0.7308, -0.2401, -0.5120,
         -0.0912, -0.2094, -0.0642,  0.2793,  0.6560,  0.2099, -0.3502,  0.0947,
          0.2870, -0.0587,  0.1992,  0.5018]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.1291, 0.0058, 0.0000, 0.0000, 0.0000, 0.0000, 0.3976,
         0.0887, 0.2234, 0.1923, 0.1938, 0.4474, 0.0274, 0.0707, 0.5935, 0.0000,
         0.1933, 0.3510],
        [0.0000, 0.0357, 0.3397, 0.1080, 0.0000, 0.0000, 0.0000, 0.0000, 0.1673,
         0.0475, 0.1926, 0.0749, 0.2884, 0.4969, 0.00

In [25]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [26]:
print(logits)

tensor([[ 0.1664, -0.2929, -0.1911, -0.3805,  0.2609,  0.0097, -0.1518,  0.3393,
          0.0936, -0.1152],
        [ 0.2844, -0.1758, -0.0117, -0.3136,  0.1866, -0.1223, -0.0946,  0.2823,
          0.2832, -0.1348],
        [ 0.3536, -0.2054, -0.1930, -0.3193,  0.1723, -0.1468, -0.1644,  0.2881,
          0.0461, -0.2152]], grad_fn=<AddmmBackward0>)


In [28]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1181, 0.0746, 0.0826, 0.0684, 0.1299, 0.1010, 0.0859, 0.1404, 0.1098,
         0.0891],
        [0.1276, 0.0805, 0.0949, 0.0702, 0.1157, 0.0850, 0.0874, 0.1273, 0.1275,
         0.0839],
        [0.1442, 0.0824, 0.0835, 0.0736, 0.1203, 0.0874, 0.0859, 0.1351, 0.1060,
         0.0816]], grad_fn=<SoftmaxBackward0>)


In [30]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0212,  0.0306,  0.0091,  ...,  0.0332, -0.0191, -0.0068],
        [-0.0170, -0.0329, -0.0157,  ...,  0.0297, -0.0117,  0.0105]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0169, 0.0279], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0287,  0.0357,  0.0269,  ...,  0.0314,  0.0381,  0.0323],
        [ 0.0228,  0.0320, -0.0133,  ..., -0.0300,  0.0288,  0.0230]],
       device='mps:0', grad_fn=<SliceBa